In [ ]:
import pandas as pd

# Load the CSV file
file_path = '/content/CQ.csv'
df = pd.read_csv(file_path)
print(df.head())

# Split the 'Lines' column by semicolon and explode the DataFrame
df['Lines'] = df['Lines'].str.split(';')
print(df.head())
df = df.explode('Lines')
print(df.head(20))


# Save the new DataFrame to a new CSV file
new_file_path = '/content/cqStations.csv'
df.to_csv(new_file_path, index=False)

#import ace_tools as tools; tools.display_dataframe_to_user(name="Expanded Subway Stations DataFrame", dataframe=df)

  Station     Lines        long        lat  Construction
0     两路口   1号线;3号线  106.549342  29.552621             0
1     石油路       1号线  106.508308  29.538110             0
2      大坪   1号线;2号线  106.519184  29.541886             0
3     富华路  18号线;9号线  106.503045  29.551102             1
4      华村       9号线  106.520043  29.550587             1
  Station        Lines        long        lat  Construction
0     两路口   [1号线, 3号线]  106.549342  29.552621             0
1     石油路        [1号线]  106.508308  29.538110             0
2      大坪   [1号线, 2号线]  106.519184  29.541886             0
3     富华路  [18号线, 9号线]  106.503045  29.551102             1
4      华村        [9号线]  106.520043  29.550587             1
   Station Lines        long        lat  Construction
0      两路口   1号线  106.549342  29.552621             0
0      两路口   3号线  106.549342  29.552621             0
1      石油路   1号线  106.508308  29.538110             0
2       大坪   1号线  106.519184  29.541886             0
2       大坪   2号线  106.519184

In [ ]:
    import requests
    import json
    from shapely import LineString
    import geopandas as gpd
    import transbigdata
    import pandas as pd
    url = 'https://map.amap.com/service/subway?_1707368894338&srhdata=3100_drw_shanghai.json'
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 '
    'Safari/537.36 Edg/121.0.0.0'
    }
    response = requests.get(url=url, headers=headers)
    result = json.loads(response.text)
    stop_data = []
    line_data = gpd.GeoDataFrame()
    names = []
    lon_lat_s = []
    for item in result['l']:
        line_name = item['kn']
        line_direction = '({start}-{end})'
        start = item['st'][0]['n']
        end = item['st'][-1]['n']

        lon_lat = []
        names.append(line_name)
        x = item['x']
        total = len(item['st'])
        num = 1

    for sub_item in item['st']:
        n_data = sub_item['n']
        sl_data = sub_item['sl']
        sl_data_parts = sl_data.split(',')
        sl_data_A = sl_data_parts[0]
        sl_data_B = sl_data_parts[1]
        lon_lat.append(sl_data_parts)
        extracted_item_0 = {
            'name': n_data,
            'linename': line_name + line_direction.format(start=start, end=end),
            'lon': sl_data_A,
            'lat': sl_data_B,
            'x': x,
            'num': num,
            'direction': 1
        }
        extracted_item_1 = {
            'name': n_data,
            'linename': line_name + line_direction.format(start=end, end=start),
            'lon': sl_data_A,
            'lat': sl_data_B,
            'x': x,
            'num': total-num+1,
            'direction': 2
        }
        stop_data.append(extracted_item_0)
        stop_data.append(extracted_item_1)
        num += 1
        lon_lat_s.append(LineString(lon_lat))
    json_output = json.dumps(stop_data, indent=4)
    with open('stop.json', 'w', encoding='utf-8') as f:
        f.write(json_output)
    stop = pd.read_json(r'stop.json', encoding='utf-8')
    stop = stop.sort_values(by=['direction', 'x', 'num'])
    stop.to_json(r'stop.json')
    line_data['name'] = names
    line_data.set_geometry(lon_lat_s, inplace=True)
    line_data.to_file("line.shp", encoding='utf-8')

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 56)

In [ ]:
import dask
import dask.dataframe as dd
import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm import tqdm


folderPath = '/content/drive/MyDrive/Ghost_Cities_Research/'
cities = ['shanghai', 'beijing', 'guangzhou', 'chongqing']
groups = [u'全部', u'18岁以下', u'60岁以上', u'低收入人群']
"""paths = ['/content/drive/MyDrive/Ghost_Cities_Research/odpf_shanghai.txt',
         '/content/drive/MyDrive/Ghost_Cities_Research/odpf_beijing.txt',
         '/content/drive/MyDrive/Ghost_Cities_Research/odpf_guangzhou.txt',
         '/content/drive/MyDrive/Ghost_Cities_Research/odpf_chongqing.txt']
"""
heatwave_dates = pd.read_csv(folderPath + 'heatwave_dates.csv')
#print(heatwave_dates.head)
city = cities[0]
group = groups[0]
dates = heatwave_dates[heatwave_dates['city']==city]['date_dt']
dates.name = None
print(dates)

#Convert large files to parquet (comment after the 1st run)
#odFilepath = folderPath + 'odpf_' + city +'.txt'
#df = dd.read_csv(odFilepath, delimiter='\t')
#df.to_parquet(folderPath + 'odpf_' + city + '(parquet)')



print('===Current City:', city, '===')
df = dd.read_parquet(folderPath + 'odpf_' + city + '(parquet)')
print(df.head())
print(df.columns)

0     20190724
1     20190725
2     20190726
3     20190727
4     20190728
5     20190729
6     20190730
7     20190805
8     20190806
9     20190807
10    20190808
11    20190809
12    20190810
13    20190811
dtype: int64
===Current City: shanghai ===
  flag   date_dt  start_grid      end_grid  pop  odleng
0   全部  20190724   310100166    3101007038  9.0     8.8
1   全部  20190724   310100166   31010044923  4.0    30.0
2   全部  20190724   310100166  310100110080  4.0    60.4
3   全部  20190724  3101001083    3101009291  4.0     4.5
4   全部  20190724  3101001083  310100133403  4.0    75.2
Index(['flag', 'date_dt', 'start_grid', 'end_grid', 'pop', 'odleng'], dtype='object')


In [ ]:
# outflows, inflows without odlength
outflows = df[df['flag']==group][['start_grid','date_dt','pop','odleng']]\
    .groupby(['start_grid', 'date_dt'])\
    .agg({'pop': 'sum'})\
    .compute().reset_index()
inflows = df[df['flag']==group][['end_grid','date_dt','pop','odleng']]\
    .groupby(['end_grid', 'date_dt'])\
    .agg({'pop': 'sum'})\
    .compute().reset_index()
print(outflows.head())
print(inflows.head())

outIDs = outflows['start_grid' ].unique()
outdf = pd.DataFrame(columns = dates, index = outIDs)
inIDs = inflows['end_grid'].unique()
indf = pd.DataFrame(columns = dates, index = inIDs)

outdf.index.name = 'start_grid'
indf.index.name = 'end_grid'
print(outdf.head())
#print(indf.head())

for index, row in outflows.iterrows():
    date = row['date_dt']
    pop = row['pop']
    gridid = row['start_grid']
    outdf.at[gridid, date] = pop
    #print(date, pop, gridid)
    #print(outdf.head())

for index, row in inflows.iterrows():
    date = row['date_dt']
    pop = row['pop']
    gridid = row['end_grid']
    indf.at[gridid, date] = pop
    #print(date, pop, gridid)
    #print(indf.head())



#newdf.reset_index(inplace=True)
print(outdf.head(10))
print(indf.head(10))
#print(newdf.loc[3101002901])
outdf['AvgPop'] = outdf.mean(axis=1)
indf['AvgPop'] = indf.mean(axis=1)
print('out:', outdf.head())
print('in:' , indf.head())
outdf.to_csv(folderPath + 'normalized_outflows_' + city + '.csv')
indf.to_csv(folderPath + 'normalized_inflows_' + city + '.csv')

   start_grid   date_dt     pop
0   310100166  20190724  4268.0
1  3101001083  20190724  1415.0
2  3101001538  20190724   392.0
3  3101001549  20190724  5735.0
4  3101001989  20190724  7159.0
       end_grid   date_dt      pop
0    3101007038  20190724   1978.0
1   31010044923  20190724   3644.0
2  310100110080  20190724   5202.0
3    3101009291  20190724   4727.0
4  310100133403  20190724  33739.0
           20190724 20190725 20190726 20190727 20190728 20190729 20190730  \
start_grid                                                                  
310100166       NaN      NaN      NaN      NaN      NaN      NaN      NaN   
3101001083      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
3101001538      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
3101001549      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
3101001989      NaN      NaN      NaN      NaN      NaN      NaN      NaN   

           20190805 20190806 20190807 20190808 20190809 20

In [ ]:
outdf.to_csv(folderPath + 'normalized_outflows_' + city + '.csv')
indf.to_csv(folderPath + 'normalized_inflows_' + city + '.csv')

In [ ]:
outflows = df[df['flag']==group][['start_grid','date_dt','pop','odleng']]\
    .groupby(['start_grid', 'date_dt'])\
    .agg({'pop': 'sum', 'odleng': 'mean'})\
    .compute().reset_index()
inflows = df[df['flag']==group][['end_grid','date_dt','pop','odleng']]\
    .groupby(['end_grid', 'date_dt'])\
    .agg({'pop': 'sum', 'odleng': 'mean'})\
    .compute().reset_index()

print(outflows.head())
print(inflows.head())
#outflows.to_csv(folderPath + 'outflows_' + group + '_' + city + '.csv')
#inflows.to_csv(folderPath + 'inflows_' + group + '_' + city + '.csv')
"""
outflowAvg = outflows.groupby(['start_grid']).mean()
inflowAvg = inflows.groupby(['end_grid', 'date_dt']).mean()

print(outflowAvg.head())
print(inflowAvg.head())
#  get average values for each start grid in outflows
"""

#outflows_dt = pd.merge(outflows, heatwave_dates, on='date_dt')
#print(outflows_dt.head())
#inflows_dt = pd.merge(inflows, heatwave_dates, on='date_dt')
#print(inflows_dt.head())

In [ ]:
outflows2 = df[df['flag']==group][['start_grid','date_dt','pop']]\
  .groupby(['start_grid', 'date_dt'])\
  .agg({'pop': 'sum'}).head()


In [ ]:
"""
citygrid = gpd.read_file(folderPath + 'Grid/' + city + '_grid.gpkg')
citygrid = citygrid[['gridid', 'geometry']]
print(citygrid.head())
citygrid['gridid'] = citygrid['gridid'].astype(int)
citygrid['num_out'] = 0
citygrid['num_in'] = 0
citygrid['len_out'] = np.ndarray(shape=(len(citygrid)))
citygrid['len_in'] = np.ndarray(shape=(len(citygrid)))
citygrid['pop'] = np.ndarray(shape=(len(citygrid)))

print(citygrid.head())
"""
"""
outflows = df[df['flag']==group][['start_grid','date_dt','pop']]\
  .groupby(['start_grid', 'date_dt'])\
  .agg({'pop': 'sum'})\
  .compute().reset_index()

inflows = df[df['flag']==group][['end_grid','date_dt','pop']]\
  .groupby(['end_grid', 'date_dt'])\
  .agg({'pop': 'sum'})\
  .compute().reset_index()

print(outflows.head())
print(inflows.head())

# outflows.head()
outflows_dt = pd.merge(outflows, heatwave_dates, on='date_dt')
print(outflows_dt.head())

inflows_dt = pd.merge(inflows, heatwave_dates, on='date_dt')
print(inflows_dt.head())
"""
"""
i = 0


for index, row in df.iterrows():
    start = row['start_grid']
    end = row['end_grid']
    #print(start, end)
    i += 1
    #print(i)
    citygrid.loc[citygrid['gridid'] == start, 'num_out'] += 1
    citygrid.loc[citygrid['gridid'] == end, 'num_in'] += 1
    citygrid.loc[citygrid['gridid'] == start, 'len_out'].append(row['len_out'])
    citygrid.loc[citygrid['gridid'] == end, 'len_in'].append(row['len_in'])
    citygrid.loc[citygrid['gridid'] == start, 'pop'].append(row['pop'])
    citygrid.loc[citygrid['gridid'] == end, 'pop'].append(row['pop'])

    if i > 1000000:
        break

citygrid.head(20)
citygrid.to_csv(folderPath + 'grid_flows' + city)"""

"""
for group in groups:
    newDf = df[df['flag'] == group[0]]
    print(newDf.head())
    newDf = newDf.drop(['flag'], axis=1)

    print(newDf.head())
    newDf['pop'] = newDf['pop'].astype('int32')
    #newDf.to_parquet(folderPath + 'odpf_' + group[1] + '_' + city)
    #newDf.to_csv(folderPath + 'odpf_' + group[1] + '_' + city)
    #newDf.to_csv(folderPath + 'odpf_' + group[1] + '_' + city + '(tsv)', sep='/t')
    break
"""
#separate different groups?
"""
#perhaps not necessary
grid = gpd.read_file(folderPath + 'Grid/' + city + '_grid.gpkg')
print(grid.head())
gridIDs = grid['gridid']
print(type(gridIDs))
print(gridIDs)

currGroup = groups[0]

cityDates = heatwave_dates[heatwave_dates['city'] == city]

groupOD = pd.DataFrame(columns = , index = gridIDs)
print(groupOD.head())
"""

# June 22 CSV Normalization


In [ ]:
import pandas as pd
import numpy as np
# John Nabors June 22, 2024
"""
Trying to normalize the dataset so that there is only one row containing each gridID
Create a new datframe with each date as a column, like a timeseries

separate data for those under 18, over 60, or in low income groups will be moved to separate files
"""
files = [('Shanghai'  , '/content/shnydx_frequency_sh_2019.csv'),
         ('Beijing'   , '/content/shnydx_frequency_bj_2019.csv'),
         ('Guangzhou' , '/content/shnydx_frequency_gz_2019.csv'),
         ('Chongqing' , '/content/shnydx_frequency_cq_2019.csv')]


for city_name, file_name in files:
    print('===Current City:', city_name, '===')
    tripFreq = pd.read_csv(file_name)

    dates   = tripFreq[' date_dt'].unique()
    gridIDs = tripFreq[' gridid' ].unique()
    flags = ['18岁以下', '60岁以上', '低收入人群', '全部']
    flagsEN = ['Under18', 'Over60', 'LowIncome', 'All']
    #print("Dates: ", dates)

    for currFlag in flags[::-1]:
        print('Current Flag:', currFlag)

        flag_filtered = tripFreq[tripFreq['flag'] == currFlag]
        newdf = pd.DataFrame(columns=dates, index = gridIDs)
        newdf.index.name = 'gridid'

        for index, row in flag_filtered.iterrows():
            date = row[' date_dt']
            tripFrequency = row[' tripfrequency']
            gridid = row[' gridid']
            #print('date: ', date, 'freq: ', tripFrequency, 'ID: ', gridid)
            newdf.at[gridid, date] = tripFrequency
        #newdf.reset_index(inplace=True)
        print(newdf.head(10))
        #print(newdf.loc[3101002901])
        newdf.to_csv('/content/' + city_name + '_Timeseries_' + flagsEN.pop() +'.csv')

